<a href="https://colab.research.google.com/github/Noroi-ui/Hackaton/blob/main/Hackathon_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Applications of Quantum Computing to Quantum Chemistry (October 7-11)

##(2) [Challenge] Ground state energy for molecule and spin system with Variational Quantum Algorithms and Trotterization.


###(a)

In [3]:
!pip install pennylane

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.2 MB/s eta 0:00:00


In [4]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem

In [5]:
# Define the symbols that describe the molecule. In this case it is H_2
symbols  = ["H","H"]

In [17]:
# Function that calculates the energy eigenvalues for given coordinates

def VQE_H2(coordinates):

  # Here it is defined molecule and hamiltonian

  molecule = qchem.Molecule(symbols,coordinates, charge=0, mult=1, basis_name='sto-3g')
  hamiltonian, qubits = qchem.molecular_hamiltonian(molecule, mapping='jordan_wigner')

  # Here it is defined the hartree-fock states
  hf = qchem.hf_state(electrons = 2,orbitals =qubits)

  # Defines circuit

  wires = qubits
  dev = qml.device("default.qubit", wires=wires)


  @qml.qnode(dev)
  def energy_exp_val(theta):
      qml.BasisState(hf, wires=range(wires))
      qml.DoubleExcitation(theta, wires=range(wires))
      return qml.expval(hamiltonian)

  #Optimizes the energy_exp_val function

  opt=qml.GradientDescentOptimizer(stepsize=0.4)
  theta=np.array(0.0, requires_grad=True)

  E = [energy_exp_val(theta)]
  angle = [theta]

  max_iterations = 75
  con_tol = 1e-08

  for n in range(max_iterations):
      theta, prev_E = opt.step_and_cost(energy_exp_val, theta)
      E.append(energy_exp_val(theta))
      angle.append(theta)

      if con_tol > abs(E[n+1]-E[n]):
          break

  return E[-1]


# The function is called

print(VQE_H2(np.array([0.0,0.0,0.0,0.0,0.0,0.7]).reshape(2,3)))

-0.8463259816606505


### (b)

In [19]:
# Hamiltonian
N = 4
def hamiltonian(h):
  H = []

  for i in range(N):
    if i == N-1:
      H -= qml.PauliZ(i) @ qml.PauliZ(0)
    else:
      H -= qml.PauliZ(i) @ qml.PauliZ(i+1)
    H -= h*qml.PauliX(i)
  return(H)

# Defines base state and device
bs = [0,0,0,0]
wires = N
dev = qml.device("default.qubit", wires=wires)

# Function that returns ground state energy for a given magnetic field

def VQE_Ising_Chain(h):

# Defines circuit
  @qml.qnode(dev)
  def energy_exp_val(theta):
    qml.BasisState(bs, wires=range(wires))

    # Here it is define an entanglement using rotation gates combined with CNOT gates.
    qml.BasicEntanglerLayers(weights=theta[0], wires=range(wires), rotation = qml.RX)
    qml.BasicEntanglerLayers(weights=theta[1], wires=range(wires), rotation = qml.RY)
    qml.BasicEntanglerLayers(weights=theta[2], wires=range(wires), rotation = qml.RZ)
    return qml.expval(hamiltonian(h))

# Optimizes circuit
  opt=qml.GradientDescentOptimizer(stepsize=0.1)
  theta=np.array([np.random.rand(1,4),np.random.rand(1,4),np.random.rand(1,4)], requires_grad=True)

  E = [energy_exp_val(theta)]
  angle = [theta]

  max_iterations = 1000
  con_tol = 1e-6

  for n in range(max_iterations):
      theta, prev_E = opt.step_and_cost(energy_exp_val, theta)
      E.append(energy_exp_val(theta))
      angle.append(theta)

      if con_tol > abs(E[n+1]-E[n]):
          break
  return E[-1]


#Function is called

print(VQE_Ising_Chain(2.3))

-9.41701834645443


### (c)

In [20]:
# Defines sub-hamiltonians
H_1 = qml.PauliX(0)@qml.PauliX(1)
H_2 = qml.PauliZ(0)@qml.PauliZ(1)

# Defines device
wires = 2
dev = qml.device('default.qubit', wires = wires)

#Defines circuit

@qml.qnode(dev)
def trotter_U(alpha, beta, t, n):
  t_n = t/n
  for i in range(n):
    qml.exp(H_1, -1j*alpha*t_n) @ qml.exp(H_2, -1j*alpha*t_n)
  return qml.probs(wires = [0,1])

# Function is called
print(trotter_U(0.9,1.0,0.4,2))

[0.87590286 0.         0.         0.12409714]
